In [ ]:
!pip install langchain
!pip install openai

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [ ]:
!pip install serpapi
!pip install google-search-results

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY"

In [ ]:
os.environ["SERPAPI_API_KEY"] = "API_KEY"

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
!pip install langchainhub

## Agent using LECL

In [ ]:
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

In [ ]:
prompt = hub.pull("hwchase17/react")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [ ]:
llm_with_stop = llm.bind(stop=["\nObservation"])

In [ ]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

In [ ]:
from langchain.agents import AgentExecutor

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"})



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"Who exactly is Vittoria Ceretti? Ceretti was born on June 7, 1998, in Brescia, Italy. Commencing her modeling journey at 14, she has already established her presence on several international catwalks. I need to calculate her age raised to the 0.43 power.
Action: Calculator
Action Input: (1998 + 14)^0.43Answer: 26.336619533501626 I now know the final answer.
Final Answer: Vittoria Ceretti is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 26.336619533501626.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Vittoria Ceretti is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 26.336619533501626."}

## Agent using ZeroShotReactAgent

In [ ]:
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"})



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Who exactly is Vittoria Ceretti? Ceretti was born on June 7, 1998, in Brescia, Italy. Commencing her modeling journey at 14, she has already established her presence on several international catwalks.
Thought: I now need to calculate her age raised to the 0.43 power.
Action: Calculator
Action Input: (1998 + 14)^0.43
Observation: Answer: 26.336619533501626
Thought: I now know the final answer.
Final Answer: Vittoria Ceretti is 26.336619533501626 years old.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': 'Vittoria Ceretti is 26.336619533501626 years old.'}

## Using a chat model

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat_model = ChatOpenAI(temperature=0)

In [ ]:
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

In [ ]:
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])

In [ ]:
# use json to enforce structure
# helps make chat model more steerable
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser

In [ ]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | chat_model_with_stop | ReActJsonSingleInputOutputParser()

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who is Leo DiCaprio's girlfriend first. Then I can calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Search",
  "action_input": "Leo DiCaprio girlfriend"
}
```
Who exactly is Vittoria Ceretti? Ceretti was born on June 7, 1998, in Brescia, Italy. Commencing her modeling journey at 14, she has already established her presence on several international catwalks.Vittoria Ceretti is Leo DiCaprio's girlfriend. Now I can calculate her current age raised to the 0.43 power.

Action:
```
{
  "action": "Calculator",
  "action_input": "1998 ^ 0.43"
}
```
Answer: 26.2576622704036Vittoria Ceretti's current age raised to the 0.43 power is 26.2576622704036.

Final Answer: Vittoria Ceretti's current age raised to the 0.43 power is 26.2576622704036.

> Finished chain.


{'input': "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
 'output': "Vittoria Ceretti's current age raised to the 0.43 power is 26.2576622704036."}